In [1]:
import os
os.environ["THEANO_FLAGS"] = "device=gpu,floatX=float32"
import theano
import theano.tensor as th

Using gpu device 0: GeForce GTX 580 (CNMeM is disabled, CuDNN not available)


In [2]:
import rnn.theano.lstm as lstm
reload(lstm)

<module 'rnn.theano.lstm' from 'rnn/theano/lstm.pyc'>

In [15]:
m = 10
b = 30
l = 100

n = 5

In [4]:
x = th.tensor3()
y0 = th.matrix()
c0 = th.matrix()
y = th.ivector()

In [5]:
layer = lstm.LSTM(m, n)

In [6]:
yh = th.nnet.softmax(layer.foldl(y0, c0, x)[0])
err = th.sum(th.nnet.categorical_crossentropy(yh, y))

In [7]:
lr = th.scalar()

glayer = th.grad(err, layer.weights)
update = [(layer.weights, layer.weights-lr*glayer)]

In [8]:
train = theano.function([y0, c0, x, y, lr], [err, yh], updates=update)
predict = theano.function([y0, c0, x], [yh])

In [9]:
import numpy as np

In [16]:
X = np.random.randn(l, b, m).astype(theano.config.floatX)
Y = np.random.randint(0, n, (b)).astype(np.int32)
y0_ = np.zeros((b,n), dtype=theano.config.floatX)
c0_ = np.zeros((b,n), dtype=theano.config.floatX)

In [17]:
predict(y0_, c0_, X)

[array([[ 0.17801453,  0.22625414,  0.15585496,  0.25424045,  0.18563591],
        [ 0.185261  ,  0.18194644,  0.18853532,  0.27313384,  0.17112339],
        [ 0.1946509 ,  0.19229355,  0.17791733,  0.30004901,  0.13508922],
        [ 0.17894717,  0.15347485,  0.24696073,  0.28369337,  0.13692389],
        [ 0.15722714,  0.14678574,  0.28056678,  0.24534455,  0.17007579],
        [ 0.16964795,  0.19446643,  0.20784672,  0.28686753,  0.14117138],
        [ 0.18922624,  0.17783906,  0.20874043,  0.26831841,  0.15587588],
        [ 0.15668611,  0.21235274,  0.22828807,  0.2427893 ,  0.15988377],
        [ 0.19309187,  0.22749022,  0.20609356,  0.24919999,  0.12412445],
        [ 0.22340846,  0.16644832,  0.18779045,  0.2553643 ,  0.16698849],
        [ 0.15513174,  0.13535428,  0.2764253 ,  0.24119923,  0.19188948],
        [ 0.19817697,  0.16263026,  0.22173324,  0.27020767,  0.1472518 ],
        [ 0.1399356 ,  0.2049952 ,  0.19378562,  0.2615639 ,  0.19971965],
        [ 0.23154528,  0.

In [18]:
print Y

[4 0 2 1 3 2 0 2 0 1 3 0 1 1 3 0 1 4 1 4 4 2 2 4 3 4 4 0 3 1]


In [42]:
iters = 1000
errs = np.zeros(iters)
for i in xrange(iters):
    err, yh = train(y0_, c0_, X, Y, 0.99999**i)
    errs[i] = err
    if i % 10 == 0:
        print '\r{:4>0}/{}, err={}'.format(i,iters, err),
print ''
print errs[0], errs[-1]
print yh
print Y

990/1000, err=24.8164558411 
25.7662658691 24.8113059998
[[ 0.12957083  0.13746351  0.1186552   0.12569484  0.48861557]
 [ 0.5474866   0.11246251  0.10019912  0.12539458  0.1144572 ]
 [ 0.19242294  0.16377382  0.30452904  0.18684906  0.15242516]
 [ 0.14766367  0.39366099  0.10266787  0.2258137   0.1301938 ]
 [ 0.13587016  0.12014776  0.1185615   0.48076296  0.14465767]
 [ 0.12042652  0.10412783  0.4530766   0.09781446  0.22455461]
 [ 0.48983553  0.13170192  0.11837449  0.12514646  0.13494164]
 [ 0.1880586   0.15625277  0.32888764  0.17520382  0.15159723]
 [ 0.50924623  0.13278016  0.10655316  0.12567276  0.12574768]
 [ 0.14450714  0.42698535  0.09308267  0.10990848  0.22551629]
 [ 0.15114671  0.15111208  0.15030198  0.39401531  0.15342396]
 [ 0.43786058  0.13492467  0.11165528  0.14558657  0.16997293]
 [ 0.14901631  0.40320969  0.09428317  0.10501931  0.24847157]
 [ 0.09655762  0.44952098  0.08505963  0.14166337  0.22719845]
 [ 0.12552445  0.11538129  0.12459189  0.51410496  0.12039746

In [43]:
print np.argmax(yh, axis=1)
sum(np.argmax(yh, axis=1)==Y)/float(len(Y))

[4 0 2 1 3 2 0 2 0 1 3 0 1 1 3 0 1 4 1 4 4 2 2 4 3 4 4 0 0 1]


0.96666666666666667